In [1]:
import pandas as pd

def txt_to_pd(input_txt): 
    mpi_processes = []
    real_time = []
    user_time = []
    sys_time = []

    # Open the text file
    with open(input_txt, 'r') as file:
        lines = file.readlines()

        # Initialize variables to hold current run and process numbers
        current_processes = None
        real = None
        user = None
        sys = None

        # Iterate through each line in the file
        for line in lines:
            if line.startswith("MPI processes:"):
                current_processes = int(line.split(":")[1].strip())
                mpi_processes.append(current_processes)
                
            elif line.startswith("real"):
                real = line.split()[-1].strip()
                minutes, seconds = real.split('m')
                seconds = seconds[:-1]
                # Convert minutes and seconds to seconds
                total_seconds = int(minutes) * 60 + float(seconds)
                real_time.append(total_seconds)
            
            elif line.startswith("user"):
                user = line.split()[-1].strip()
                minutes, seconds = user.split('m')
                seconds = seconds[:-1]
                # Convert minutes and seconds to seconds
                total_seconds = int(minutes) * 60 + float(seconds)
                user_time.append(total_seconds)
                
            
            elif line.startswith("sys"):
                sys = line.split()[-1].strip()
                minutes, seconds = sys.split('m')
                seconds = seconds[:-1]
                # Convert minutes and seconds to seconds
                total_seconds = int(minutes) * 60 + float(seconds)
                sys_time.append(total_seconds)

    data = {
    'MPI processes': mpi_processes,
    'Real time': real_time,
    'User time': user_time,
    'System time': sys_time}

    data = pd.DataFrame(data)

    return data


# MPI scaling: 
plotting the execution time from 2 processes up to 256 processes

In [2]:
df = txt_to_pd('mpi_scaling.txt')
df

,MPI processes,Real time,User time,System time
0,2,1565.986,3117.178,1.431
1,4,733.536,2899.800,2.358
2,6,403.249,2384.972,3.700
3,8,477.725,3766.183,5.679
4,10,327.489,3216.891,7.314
...,...,...,...,...
123,248,37.150,3713.272,272.548
124,250,36.377,3617.018,279.145
125,252,37.071,3688.596,266.279
126,254,36.365,3613.696,271.832


In [7]:
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

# Plot the execution time 

fig = px.line(df, x='MPI processes', y='Real time', title="MPI Scaling")
total_processes = df['MPI processes'].max()
quarter_points = [int(total_processes / 4) * i for i in range(5)]  # [0, 1/4, 2/4, 3/4, 1]

# Set the x-axis tick marks and labels
fig.update_xaxes(
    tickvals=quarter_points,
    ticktext=[f'{int(p)}' for p in quarter_points]  # You can customize the format of the labels
)

# Annotate the quarter points with y-axis labels
for x in quarter_points:
    # Find the nearest MPI processes value to the quarter point
    nearest_idx = np.abs(df['MPI processes'] - x).idxmin()
    nearest_mpi_processes = df.loc[nearest_idx, 'MPI processes']
    nearest_real_time = df.loc[nearest_idx, 'Real time']
    fig.add_annotation(x=nearest_mpi_processes, y=nearest_real_time, text=str(nearest_real_time), showarrow=True, arrowhead=1)

# Show the plot
fig.show()

IndexError: index 0 is out of bounds for axis 0 with size 0